In [26]:
import os
import csv
from datetime import datetime, timezone
import influxdb_client
from influxdb_client.client.write_api import ASYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from influxdb_client import Point
from urllib3.exceptions import NewConnectionError

INFLUX_URL = "http://influxdb2:8086"
INFLUX_TOKEN = "MyInitialAdminToken0=payosno"

client = None
write_api = None

try:
    client = influxdb_client.InfluxDBClient(
        url=INFLUX_URL,
        token=INFLUX_TOKEN,
        org="docs"
    )

    write_api = client.write_api(
        batch=5000,
        write_api=ASYNCHRONOUS,
    )
    
except (InfluxDBError, NewConnectionError) as e:
    print("[ERROR] Error al conectar con InfluxDB:")
    print(f"   Detalle: {e}")



ficheros = os.listdir("./UT02/data")

for fic in ficheros:
    with open("./UT02/data/"+fic, newline="", encoding="utf-8") as f:
        reader = csv.reader(f)

        next(reader, None)

        for row in reader:
            name = row[1]
            symbol = row[2]

            date_ = row[3]
            dt = datetime.strptime(date_,"%Y-%m-%d %H:%M:%S")
            dt_utc = dt.replace(tzinfo=timezone.utc)
            time_date = dt_utc.isoformat().replace('+00:00','Z')

            high = float(row[4])
            low = float(row[5])
            close_ = float(row[6])
            open_ = float(row[7])
            volume = float(row[8])
            marketcap = float(row[9])

        p = Point("daily_quote") \
            .tag("Symbol", symbol) \
            .tag("Name", name) \
            .time(time_date) \
            .field("Close", close_) \
            .field("High", high) \
            .field("Low", low) \
            .field("Open", open_) \
            .field("Volume", volume) \
            .field("Marketcap", marketcap)
        
        write_api.write(bucket="crypto_raw", org="docs", record=p)

write_api.close()


